## C_l sampler initial notes and tests

In [1]:
import os
import glob
import sys

import numpy as np
import healpy as hp

from scipy import signal
from scipy.fftpack import fft, fft2, fftshift, fftfreq
from scipy import integrate
from scipy.stats import invgamma

from pygdsm import GlobalSkyModel16
from pygdsm import GlobalSkyModel


# Plotting
#import matplotlib.pyplot as plt
import cmocean
import cmocean.cm as cmo
import seaborn as sns
import pylab as plt
import matplotlib as mpl
import matplotlib.pyplot as pyplot
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition, mark_inset)
from matplotlib import ticker
# tango colors for colorpairs
sys.path.append("/Users/user/Documents/hera/tango-colors")
# from tango_colors import Tango
# tango = Tango('HTML')
import corner


# HYDRA (for HERA antennas)
sys.path.append("/Users/user/Documents/hera/Hydra") # change this to your own path
import hydra
from hydra.utils import build_hex_array

In [2]:
plt.rcParams['font.size'] = '18'
plt.rcParams['savefig.facecolor']='white'
plt.rcParams['axes.titlepad'] = 16
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif"
})
plt.rcParams["errorbar.capsize"] = 5

# Function definitions

## Mode conversion functions

### `healpy2alms`

In [3]:
def healpy2alms(healpy_modes):
    """
    Takes a complex array of alms (positive modes only) and turns into
    a real array split as [real, imag] making sure to remove the 
    m=0 modes from the imag-part.
      
     Parameters
    ----------
    * healpy_modes (ndarray (complex)):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
    
    Returns
    -------
    * alms (ndarray (floats))
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and is split into a real (first) and imag (second) part. The
            Imag part is smaller as the m=0 modes shouldn't contain and 
            imaginary part. 
    """
    lmax = hp.sphtfunc.Alm.getlmax(healpy_modes.size) # to remove the m=0 imag modes
    alms = np.concatenate((healpy_modes.real,healpy_modes.imag[(lmax+1):]))
        
    return alms    


### `alms2healpy`

In [4]:
def alms2healpy(alms, lmax):
    """
    Takes a real array split as [real, imag] (without the m=0 modes 
    imag-part) and turns it into a complex array of alms (positive 
    modes only) ordered as in HEALpy.
      
     Parameters
    ----------
    * alms (ndarray (floats))
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.

    
    Returns
    -------
    * healpy_modes (ndarray (complex)):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
            
    """
    
    real_imag_split_index = int((np.size(alms)+(lmax+1))/2)
    real = alms[:real_imag_split_index]
    
    add_imag_m0_modes = np.zeros(lmax+1)
    imag = np.concatenate((add_imag_m0_modes, alms[real_imag_split_index:]))
    
    healpy_modes = real + 1.j*imag
    
    return healpy_modes

### `lm_order_alms`

In [5]:
def lm_order_alms(alms, lmax):
    """
    Takes a real array split as [real, imag] (without the m=0 modes 
    imag-part) ordered as (m,l) and reorders as (l,m) (still omitting
    the m=0 imaginary modes)
      
     Parameters
    ----------
    * alms (ndarray (floats))
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.

    * lmax (integer)
            The maximum ell-value 

    
    Returns
    -------
    * healpy_modes (ndarray (complex)):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
            
    """
    
    healpy_modes = alms2healpy(alms,lmax)

    # intialise for-loop
    real_modes = np.zeros_like(healpy_modes.real)
    imag_modes = np.zeros_like(healpy_modes.imag[(lmax+1):])
    real_idx = 0
    imag_idx = 0
    
    for ell in np.arange(0,lmax+1):
        for em in np.arange(0,ell+1): 
            healpy_idx = hp.sphtfunc.Alm.getidx(lmax, ell, em)
            real_modes[real_idx] = healpy_modes[healpy_idx].real
            real_idx +=1
            
            if em != 0:
                imag_modes[imag_idx] = healpy_modes[healpy_idx].imag
                imag_idx +=1

    return np.concatenate((real_modes, imag_modes))


### `get_em_ell_idx`

In [6]:
def get_em_ell_idx(lmax):
    """
    Function to get the em, ell, and index of all the modes given the lmax. 
    (m,l)-ordering, (m-major ordering)

    Parameters
    ----------
    * lmax: (int)
        Maximum ell value for alms

    Returns
    -------
    * ems: (list (int))
        List of all the em values of the alms (m,l)-ordering (m-major)

    * ells: (list (int))
        List of all the ell values of the alms (m,l)-ordering (m-major)
        
    * idx: (list (int)) 
        List of all the indices for the alms

    """

    ells_list = np.arange(0,lmax+1)
    em_real = np.arange(0,lmax+1)
    em_imag = np.arange(1,lmax+1)
    
    Nreal = 0
    i = 0
    idx = []
    ems = []
    ells = []

    for em in em_real:
        for ell in ells_list:
            if ell >= em:
                idx.append(i)
                ems.append(em)
                ells.append(ell)
                
                Nreal += 1
                i +=1
    
    Nimag=0

    for em in em_imag:
        for ell in ells_list:
            if ell >= em:
                idx.append(i)
                ems.append(em)
                ells.append(ell)

                Nimag += 1
                i += 1

    return ems, ells, idx

### `find_common_true_index`

In [7]:
def find_common_true_index(arr_em, arr_ell, lmax):
    """
    Find the common index between two arrays of same length consisting of true and false.

    Parameters
    ----------
    * arr_em: (ndarray (boolean))
        The m array to compare, consisting of true and false

    * arr_ell: (ndarray (boolean))
        The ell array to compare, consisting of true and false

    Returns
    -------
    * idx_real: (int)
        The common index for the real part

    * idx_imag: (int)
        The common index for the imag part

    """
    real_imag_split_index = int(((lmax+1)**2 + (lmax+1))/2)

    real_idx = []
    imag_idx = []

    for idx in range(len(arr_em)):
        if arr_em[idx] and arr_ell[idx] and idx < real_imag_split_index:
            real_idx = idx
        elif arr_em[idx] and arr_ell[idx] and idx >= real_imag_split_index:
            imag_idx = idx

    return real_idx, imag_idx


### `get_idx_ml`

In [8]:
def get_idx_ml(em, ell, lmax):
    """
    Get the global index for the alms (m,l)-ordering (m-major) given a m 
    and ell value. 
    
    Parameters
    ----------
    * em: (int)
        The em value of the mode. Note, em cannot be greater than the ell value.

    * ell: (int)
        The ell value of the mode. Note, ell has to be larger or equal to the em value.

    * lmax: (int)
        The lmax of the modes

    Returns
    -------
    * common_idx_real: (int)
        The global index of the real part of the spherical harmonic mode

    * common_idx_imag: (int)
        The global index of the imaginary part of the spherical harmonic mode. 
        There are no m=0 imaginary mode, so in case of m=0 it returns and empty list [].

    """

    assert np.all(em <= ell), "m cannot be greater than the ell value"
    ems_idx, ells_idx, idx = get_em_ell_idx(lmax)

    em_check = np.array(ems_idx) == em
    ell_check = np.array(ells_idx) == ell

    common_idx_real, common_idx_imag = find_common_true_index(arr_em=em_check,
                                                              arr_ell=ell_check,
                                                              lmax=lmax)
    if common_idx_imag == []:
        idx_list = [common_idx_real]
    else:
        idx_list = [common_idx_real, common_idx_imag]
        
    for common_idx in idx_list:
        assert common_idx == idx[common_idx], "the global index does not match the index list"
        assert em == ems_idx[common_idx], "The em corresponding to the global index does not match the chosen em"
        assert ell == ells_idx[common_idx], "The ell corresponding to the global index does not match the vhosen ell"

    return common_idx_real, common_idx_imag

### `get_em_labels`

In [9]:
def get_em_labels(lmax):
    ells = np.arange(0,lmax+1)
    em_real = np.arange(0,lmax+1) 
    em_imag = np.arange(1,lmax+1)

    # ylabel = []
    # First append all real (l,m) values
    Nreal = 0
    i = 0
    idx = []
    ems = []
    for em in em_real:
        for ell in ells:
            if ell >= em:
                # ylabel.append((ell, em))
                if ell == em:
                    idx.append(i)
                    if ell == 0:
                        ems.append(fr'$m= {em}$')
                    else:
                        ems.append(fr'${em}$')
                Nreal += 1
                i += 1

    # Then all imaginary -- note: no m=0 modes!  
    Nimag = 0
    for em in em_imag:
        for ell in ells:
            if ell >= em:
                # ylabel.append((ell,em))
                if ell == em:
                    idx.append(i)
                    ems.append(fr'${em}$')
                Nimag += 1
                i += 1
    return ems, idx

### `get_healpy_from_gsm`

In [10]:
def get_healpy_from_gsm(freq, lmax, nside=64, resolution="low", output_model=False, output_map=False):
    """
    Generate an array of alms (HEALpy ordered) from gsm 2016 (https://github.com/telegraphic/pygdsm)
    
    Parameters
    ----------
    * freqs: (float or np.array)
        Frequency (in MHz) for which to return GSM model
        
    * lmax: (int)
        Maximum l value for alms
        
    * nside: (int)
        The NSIDE you want to upgrade/downgrade the map to. Default is nside=64.

    * resolution: (str)
        if "low/lo/l":  The GSM nside = 64  (default)
        if "hi/high/h": The GSM nside = 1024 

    * output_model: (Boolean) optional
        If output_model=True: Outputs model generated from the GSM data. 
        If output_model=False (default): no model output.

    * output_map: (Boolean) optional
        If output_map=True: Outputs map generated from the GSM data. 
        If output_map=False (default): no map output.

    Returns
    -------
    *healpy_modes: (np.array)
        Complex array of alms with same size and ordering as in healpy (m,l)
    
    *gsm_2016: (PyGDSM 2016 model) optional
        If output_model=True: Outputs model generated from the GSM data. 
        If output_model=False (default): no model output.

    *gsm_map: (healpy map) optional
        If output_map=True: Outputs map generated from the GSM data. 
        If output_map=False (default): no map output.
    
    """
    gsm_2016 = GlobalSkyModel16(freq_unit='MHz', resolution=resolution) 
    gsm_map = gsm_2016.generate(freqs=freq)
    gsm_upgrade = hp.ud_grade(gsm_map, nside)
    healpy_modes_gal = hp.map2alm(maps=gsm_upgrade,lmax=lmax)

    # Per default it is in gal-coordinates, convert to equatorial
    rot_gal2eq = hp.Rotator(coord="GC")
    healpy_modes_eq = rot_gal2eq.rotate_alm(healpy_modes_gal)

    if output_model == False and output_map == False: # default
        return healpy_modes_eq
    elif output_model == False and output_map == True:
        return healpy_modes_eq, gsm_map 
    elif output_model == True and output_map == False:
        return healpy_modes_eq, gsm_2016 
    else:
        return healpy_modes_eq, gsm_2016, gsm_map

# Main

In [17]:
# Display available folders
parentpath = '/Users/user/Documents/hera/cl_sampler_analysis/data/'
[file[52:] for file in sorted(glob.glob(parentpath+'*'))]

['cl_sampler_100']

In [19]:
# Set paths and create figure folder within data folder
path = parentpath+'cl_sampler_100/'
figpath = path+'figures/'
if not os.path.isdir(figpath): os.makedirs(figpath)

In [21]:
# Load precomputed data and see which are available
try:
    precomp = np.load(path+'precomputed_data.npz')
except:
    precomp = np.load(path+'precomputed_data_20_1.npz')
precomp.files

['vis_response',
 'autos',
 'x_true',
 'inv_noise_cov',
 'min_prior_std',
 'inv_prior_cov',
 'a_0',
 'data_seed',
 'prior_seed',
 'incl_cosmic_var',
 'wf_soln',
 'nside',
 'lmax',
 'ants',
 'beam_diameter',
 'freqs',
 'lsts_hours',
 'precomp_time',
 'avg_iter_time',
 'total_time']

In [22]:
# Assign precomputed data 
vis_response = precomp['vis_response']
autos = precomp['autos']
x_true = precomp['x_true']
inv_noise_cov = precomp['inv_noise_cov']
min_prior_std = precomp['min_prior_std']
inv_prior_cov = precomp['inv_prior_cov']
a_0 = precomp['a_0']
data_seed = precomp['data_seed']
prior_seed = precomp['prior_seed']
wf_soln = precomp['wf_soln']
nside = precomp['nside']
lmax = precomp['lmax']
beam_diameter = precomp['beam_diameter']
freqs = precomp['freqs']
lsts_hours = precomp['lsts_hours']
precomp_time = precomp['precomp_time']
avg_iter_time = precomp['avg_iter_time']
total_time = precomp['total_time']

In [23]:
# Anttenas had to be in a separate file
if os.path.isfile(path+'ant_pos.npz'):
    ant_pos = np.load(path+'ant_pos.npz')
    print(ant_pos.files)
else: 
    # Assume its the (3,4,3) setup:
    ant_pos = build_hex_array(hex_spec=(3,4), d=14.6)
    print(r'`ant_pos` assumed to be in (3,4,3) position, 10 antennas')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


## Get keys for results

### alms

In [46]:
# Load in one result file to get keys
alm_file_list = glob.glob(path+'alms_*')
alm_key_file = np.load(alm_file_list[0])
alm_key_file.files

['omega_0',
 'omega_1',
 'alm_random_seed',
 'x_soln',
 'rhs',
 'convergence_info',
 'solver_time',
 'iteration_time']

### C_ell

In [28]:
cl_file_list = glob.glob(path+'cls_*')
cl_key_file = np.load(cl_file_list[0])
cl_key_file.files

['sigma_ell', 'cl_samples', 'cl_random_seed']

## Assign results

In [36]:
# Allocate arrays
x_solns = np.empty((len(alm_file_list),len(alm_key_file['x_soln'])))
sigma_ells = np.empty((len(cl_file_list),len(cl_key_file['sigma_ell'])))
cl_samples = np.empty((len(cl_file_list),len(cl_key_file['cl_samples'])))

In [51]:
# Load results and assign
alm_file_list.sort()
key = 0
for filename in alm_file_list:
    result = np.load(filename)
    omega_0 = result['omega_0']
    omega_1 = result['omega_1']
    rhs = result['rhs']
    
    # Notify if any realisations didn't converge
    convergence_info = result['convergence_info']
    if np.any(convergence_info): print(convergence_info)
    
    # Combine solutions to array
    x_soln = result['x_soln']
    x_solns[key] = x_soln

    key += 1

In [ ]:
cl_file_list.sort()
key = 0
for filename in cl_file_list:
    result = np.load(filename)
    
    sigma_ell = result['sigma_ell']
    cl_sample = result['cl_samples']

    
    # Combine solutions to array
    sigma_ells[key] = sigma_ell

    key += 1